# House Construction Expense Tracker

This notebook helps you track and analyze expenses for your house construction project. You can:
1. Record expenses with categories and subcategories
2. View total expenses by category
3. Track spending patterns over time
4. Visualize expense distribution
5. Auto-save data to CSV and Excel
6. Create automatic backups

All data is stored securely in the `house_expenses_data` folder.

## 1. Import Required Libraries

First, let's import the necessary Python libraries for data management and visualization.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import shutil

# Set the style for better visualizations
sns.set_style("whitegrid")
sns.set_palette("husl")

## 2. Setup Expense Categories and Data Storage

Define the expense categories and setup data storage structure.

In [ ]:
import os
import pandas as pd

# Define the data directory structure
DATA_DIR = 'house_expenses_data'
CSV_FILE = os.path.join(DATA_DIR, 'expenses.csv')
EXCEL_FILE = os.path.join(DATA_DIR, 'house_expenses.xlsx')
BACKUP_DIR = os.path.join(DATA_DIR, 'backups')

# Create necessary directories
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(BACKUP_DIR, exist_ok=True)

# Define expense categories and subcategories
expense_categories = {
    'Planning': ['House Plan', 'Other Expenses'],
    'House Plot': ['Land Cost', 'Land Registration'],
    'Building Plan Approval': ['EC market value', 'Property Tax'],
    'Initial Setup': ['Basic Things', 'Construction Equipment'],
    'Transportation': ['Transportation', 'Coolie (Wood)', 'Permits', 'Personal Transport'],
    'Building Material': {
        'Wood': ['Material (Wood)'],
        'Stone': ['RR Stones', 'Basement Stones', 'kankara', 'Ramapuram Kankara'],
        'Sand': ['Sand'],
        'Iron': ['Iron', 'Binding wire'],
        'Cement': ['Cement'],
        'Bricks': ['Bricks']
    },
    'Construction': ['Coolie', 'Labour charges', 'JCB', 'Rent for Miller & rekulu'],
    'Doors & Windows': ['Carpenter'],
    'Pooja': ['Bhumi Pooja']
}

# Create an empty DataFrame to store expenses
expenses_df = pd.DataFrame(columns=[
    'Date',
    'Category',
    'Subcategory',
    'Description',
    'Amount',
    'Payment_Method',
    'Receipt_Reference'
])

# Print available categories and subcategories
print("Available Expense Categories:")
for category, subcategories in expense_categories.items():
    print(f"\n{category}:")
    for sub in subcategories:
        print(f"  - {sub}")

Available Expense Categories:

Foundation:
  - Excavation
  - Concrete
  - Waterproofing
  - Other

Structure:
  - Framing
  - Roof
  - Windows
  - Doors
  - Other

Systems:
  - Electrical
  - Plumbing
  - HVAC
  - Solar
  - Other

Interior:
  - Flooring
  - Walls
  - Ceiling
  - Paint
  - Lighting
  - Other

Exterior:
  - Siding
  - Landscaping
  - Driveway
  - Fencing
  - Other

Kitchen:
  - Cabinets
  - Countertops
  - Appliances
  - Plumbing Fixtures
  - Other

Bathroom:
  - Fixtures
  - Tiles
  - Vanities
  - Shower/Tub
  - Other

Permits:
  - Building Permit
  - Inspection Fees
  - Other

Labor:
  - Contractor Fees
  - Specialized Labor
  - General Labor
  - Other

Other:
  - Materials
  - Tools
  - Rentals
  - Miscellaneous


## 3. Data Management Functions

Create functions to:
- Add new expenses
- Save data to CSV and Excel
- Load existing data
- Create automatic backups

In [4]:
def save_data():
    """
    Save the expense data to both CSV and Excel files with automatic backup
    """
    global expenses_df
    
    # Save to CSV
    expenses_df.to_csv(CSV_FILE, index=False)
    
    # Save to Excel with formatting
    with pd.ExcelWriter(EXCEL_FILE, engine='openpyxl') as writer:
        expenses_df.to_excel(writer, index=False, sheet_name='Expenses')
    
    # Create backup with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_file = os.path.join(BACKUP_DIR, f'expenses_backup_{timestamp}.csv')
    shutil.copy2(CSV_FILE, backup_file)

def load_data():
    """
    Load existing expense data from CSV file
    """
    global expenses_df
    
    if os.path.exists(CSV_FILE):
        expenses_df = pd.read_csv(CSV_FILE)
        expenses_df['Date'] = pd.to_datetime(expenses_df['Date'])
        print("Loaded existing data successfully.")
        return True
    else:
        print("No existing data file found. Starting with empty dataset.")
        return False

def add_expense(date, category, subcategory, description, amount, payment_method, receipt_reference):
    """
    Add a new expense to the dataset
    """
    global expenses_df
    
    # Validate category and subcategory
    if category not in expense_categories:
        raise ValueError(f"Invalid category. Must be one of: {list(expense_categories.keys())}")
    if subcategory not in expense_categories[category]:
        raise ValueError(f"Invalid subcategory for {category}. Must be one of: {expense_categories[category]}")
    
    # Create new expense entry
    new_expense = pd.DataFrame({
        'Date': [pd.to_datetime(date)],
        'Category': [category],
        'Subcategory': [subcategory],
        'Description': [description],
        'Amount': [float(amount)],
        'Payment_Method': [payment_method],
        'Receipt_Reference': [receipt_reference]
    })
    
    # Append to existing data
    expenses_df = pd.concat([expenses_df, new_expense], ignore_index=True)
    
    # Save updated data
    save_data()
    print(f"Added new expense: {description} - ${amount:,.2f}")

# Load existing data if available
load_data()

Loaded existing data successfully.


True

In [ ]:
def clear_data():
    """Clear all existing expense data"""
    global expenses_df
    expenses_df = pd.DataFrame(columns=[
        'Date',
        'Category',
        'Subcategory',
        'Description',
        'Amount',
        'Payment_Method',
        'Receipt_Reference'
    ])
    save_data()
    print("All existing data cleared.")

def add_multiple_expenses(expenses_list):
    """
    Add multiple expenses at once
    expenses_list: list of dictionaries containing expense details
    """
    for expense in expenses_list:
        add_expense(
            date=expense['date'],
            category=expense['category'],
            subcategory=expense['subcategory'],
            description=expense['description'],
            amount=expense['amount'],
            payment_method=expense.get('payment_method', 'Not Specified'),
            receipt_reference=expense.get('receipt_reference', '')
        )
    print(f"Added {len(expenses_list)} expenses successfully.")

# Clear existing data
clear_data()

# Prepare the expense data
expenses = [
    {"date": "2025-03-05", "category": "Planning", "subcategory": "House Plan", "amount": 3500, "description": "plan design by zara"},
    {"date": "2024-10-05", "category": "House Plot", "subcategory": "Land Cost", "amount": 2500000, "description": "cost for the land"},
    {"date": "2024-10-05", "category": "House Plot", "subcategory": "Land Registration", "amount": 60000, "description": "registration for the land"},
    {"date": "2025-03-20", "category": "Building Plan Approval", "subcategory": "EC market value", "amount": 700, "description": "sub-register office"},
    {"date": "2025-03-22", "category": "Building Material", "subcategory": "Material (Wood)", "amount": 701000, "description": "bought teak"},
    {"date": "2025-04-01", "category": "Transportation", "subcategory": "Transportation", "amount": 4000, "description": "transportation of teak to home"},
    {"date": "2025-04-01", "category": "Transportation", "subcategory": "Coolie (Wood)", "amount": 4000, "description": "Coolie charges for teak"},
    {"date": "2025-04-01", "category": "Transportation", "subcategory": "Permits", "amount": 600, "description": "check post cost"},
    {"date": "2025-04-01", "category": "Transportation", "subcategory": "Permits", "amount": 2600, "description": "Permit for transportation of teak"},
    {"date": "2025-04-01", "category": "Transportation", "subcategory": "Personal Transport", "amount": 500, "description": "General transportation to pormamilla"},
    {"date": "2025-04-01", "category": "Planning", "subcategory": "Other Expenses", "amount": 500, "description": "Given to Poojari"},
    {"date": "2025-04-13", "category": "Construction", "subcategory": "Coolie", "amount": 10000, "description": "Advance for the Coolie, construction"},
    {"date": "2025-04-21", "category": "Building Plan Approval", "subcategory": "Property Tax", "amount": 8197, "description": "property tax"},
    {"date": "2025-04-23", "category": "Initial Setup", "subcategory": "Basic Things", "amount": 1000, "description": "Electrical & water connections"},
    {"date": "2025-04-26", "category": "Building Material", "subcategory": "RR Stones", "amount": 4000, "description": "1 tractor"},
    {"date": "2025-04-26", "category": "Initial Setup", "subcategory": "Construction Equipment", "amount": 2700, "description": "bought 10 tubs, 2 paralu"},
    {"date": "2025-04-26", "category": "Construction", "subcategory": "Bhumi Pooja", "amount": 1530, "description": "vastralu"},
    {"date": "2025-04-09", "category": "Construction", "subcategory": "Carpenter", "amount": 4000, "description": "Advance for Aachari"},
    {"date": "2025-04-28", "category": "Construction", "subcategory": "Coolie", "amount": 100000, "description": "first payment to masthan, after marking"},
    {"date": "2025-04-28", "category": "Construction", "subcategory": "Bhumi Pooja", "amount": 3000, "description": "Bought things for Bhumi Pooja"},
    {"date": "2025-04-29", "category": "Construction", "subcategory": "Ramapuram Kankara", "amount": 23200, "description": "6 units kankara before bhumi pooja"},
    {"date": "2025-05-06", "category": "Construction", "subcategory": "JCB", "amount": 16000, "description": "rent for JCB"},
    {"date": "2025-05-01", "category": "Construction", "subcategory": "kankara", "amount": 4800, "description": "1 tractor 40mm kankara"},
    {"date": "2025-05-01", "category": "Construction", "subcategory": "Sand", "amount": 8000, "description": "4 tractors sand"},
    {"date": "2025-05-06", "category": "Construction", "subcategory": "Sand", "amount": 4600, "description": "2 tractors sand"},
    {"date": "2025-05-06", "category": "Construction", "subcategory": "Basement Stones", "amount": 28100, "description": "7 tractors stones"},
    {"date": "2025-05-05", "category": "Construction", "subcategory": "Rent for Miller & rekulu", "amount": 5600, "description": "rent for miller & rekulu"},
    {"date": "2025-05-06", "category": "Construction", "subcategory": "Coolie", "amount": 1500, "description": "side charges for 2 extra persons to dug pits"},
    {"date": "2025-04-30", "category": "Construction", "subcategory": "Bhumi Pooja", "amount": 10000, "description": "Expenses on the day of Bhumi Pooja"},
    {"date": "2025-05-02", "category": "Initial Setup", "subcategory": "Basic Things", "amount": 1500, "description": "Pipe for water"},
    {"date": "2025-05-04", "category": "Initial Setup", "subcategory": "Basic Things", "amount": 1200, "description": "wire for vibration"},
    {"date": "2025-05-10", "category": "Construction", "subcategory": "Basement Stones", "amount": 20000, "description": "5 tractor stones"},
    {"date": "2025-05-09", "category": "Construction", "subcategory": "Labour charges", "amount": 2000, "description": "Labour charge while recertification of iron"},
    {"date": "2025-05-09", "category": "Construction", "subcategory": "Iron", "amount": 95000, "description": "Total iron cost 310677, paid first installement through PhonePe"},
    {"date": "2025-05-10", "category": "Construction", "subcategory": "Coolie", "amount": 1000, "description": "Gave to Mestri while gng to Delhi, for general expenditure"},
    {"date": "2025-05-13", "category": "Construction", "subcategory": "Basement Stones", "amount": 12000, "description": "3 trips stones"},
    {"date": "2025-05-16", "category": "Construction", "subcategory": "Coolie", "amount": 1000, "description": "general expenses"},
    {"date": "2025-05-19", "category": "Construction", "subcategory": "Coolie", "amount": 5000, "description": "front side basement guntha with coolies"},
    {"date": "2025-05-15", "category": "Construction", "subcategory": "Iron", "amount": 55000, "description": "Iron installement second"},
    {"date": "2025-05-20", "category": "Construction", "subcategory": "Iron", "amount": 60677, "description": "Third installement"},
    {"date": "2025-05-20", "category": "Construction", "subcategory": "Cement", "amount": 50000, "description": "Ramco cement first installment"},
    {"date": "2025-05-23", "category": "Construction", "subcategory": "Coolie", "amount": 40000, "description": "Gave to mastan; Phonepe"},
    {"date": "2025-06-01", "category": "Construction", "subcategory": "Coolie", "amount": 50000, "description": "Gave to kamal basha; cred"},
    {"date": "2025-05-24", "category": "Construction", "subcategory": "kankara", "amount": 8000, "description": "1 tractor kankara"},
    {"date": "2025-05-26", "category": "Construction", "subcategory": "Ramapuram Kankara", "amount": 13700, "description": "3 units"},
    {"date": "2025-05-09", "category": "Construction", "subcategory": "Sand", "amount": 4600, "description": "2 tractors sand"},
    {"date": "2025-05-22", "category": "Construction", "subcategory": "Sand", "amount": 4600, "description": "2 tractors sand"},
    {"date": "2025-05-28", "category": "Construction", "subcategory": "Sand", "amount": 2000, "description": "1 tractor"},
    {"date": "2025-05-29", "category": "Construction", "subcategory": "Sand", "amount": 4000, "description": "2 tractors sand"},
    {"date": "2025-05-31", "category": "Construction", "subcategory": "Sand", "amount": 4000, "description": "2 tractors sand"},
    {"date": "2025-05-28", "category": "Construction", "subcategory": "Rent for Miller & rekulu", "amount": 2400, "description": "rent"},
    {"date": "2025-05-29", "category": "Construction", "subcategory": "Rent for Miller & rekulu", "amount": 6000, "description": "rent"},
    {"date": "2025-05-24", "category": "Construction", "subcategory": "Coolie (extra charges)", "amount": 1200, "description": "cost for food, while doing plinth beam"},
    {"date": "2025-05-22", "category": "Construction", "subcategory": "Iron", "amount": 3100, "description": "Binding wire"},
    {"date": "2025-05-29", "category": "Construction", "subcategory": "Iron", "amount": 75000, "description": "Iron ; phonepe"},
    {"date": "2025-05-29", "category": "Construction", "subcategory": "Cement", "amount": 20000, "description": "cement; Phonepe"},
    {"date": "2025-05-21", "category": "Construction", "subcategory": "Bricks", "amount": 6000, "description": "fly ash bricks"}
]

# Add all expenses
add_multiple_expenses(expenses)

## 4. Expense Analysis and Visualization

Functions to analyze expenses and create visualizations for better insights.

In [ ]:
# Clear existing data
expenses_df = pd.DataFrame(columns=[
    'Date',
    'Category',
    'Subcategory',
    'Description',
    'Amount',
    'Payment_Method',
    'Note'
])

def bulk_add_expenses(expenses_data):
    """
    Add multiple expenses at once
    expenses_data: List of dictionaries containing expense details
    """
    global expenses_df
    
    for expense in expenses_data:
        # Create new expense entry
        new_expense = pd.DataFrame({
            'Date': [pd.to_datetime(expense['date'])],
            'Category': [expense['category']],
            'Subcategory': [expense['subcategory']],
            'Description': [expense['description']],
            'Amount': [float(expense['amount'])],
            'Payment_Method': [expense.get('payment_method', 'Not specified')],
            'Note': [expense['note']]
        })
        
        # Append to existing data
        expenses_df = pd.concat([expenses_df, new_expense], ignore_index=True)
    
    # Save updated data
    save_data()
    print(f"Added {len(expenses_data)} expenses successfully.")

# Define all expenses
expenses_data = [
    {
        'date': '2024-10-05',
        'category': 'House Plot',
        'subcategory': 'Land Cost',
        'description': 'Land purchase',
        'amount': 2500000,
        'note': 'cost for the land'
    },
    {
        'date': '2024-10-05',
        'category': 'House Plot',
        'subcategory': 'Land Registration',
        'description': 'Land registration fees',
        'amount': 60000,
        'note': 'registration for the land'
    },
    {
        'date': '2025-03-05',
        'category': 'Planning',
        'subcategory': 'House Plan',
        'description': 'House plan design',
        'amount': 3500,
        'payment_method': 'Cash',
        'note': 'plan design by zara'
    },
    {
        'date': '2025-03-20',
        'category': 'Building Plan Approval',
        'subcategory': 'EC market value',
        'description': 'EC approval',
        'amount': 700,
        'note': 'sub-register office'
    },
    {
        'date': '2025-03-22',
        'category': 'Building Material',
        'subcategory': 'Material (Wood)',
        'description': 'Teak wood purchase',
        'amount': 701000,
        'note': 'bought teak'
    }
]

# Add more expenses here...
more_expenses = [
    {
        'date': '2025-04-01',
        'category': 'Transportation',
        'subcategory': 'Transportation',
        'description': 'Teak transportation',
        'amount': 4000,
        'note': 'transportation of teak to home'
    },
    {
        'date': '2025-04-01',
        'category': 'Transportation',
        'subcategory': 'Coolie (Wood)',
        'description': 'Labor charges for teak',
        'amount': 4000,
        'note': 'Coolie charges for teak'
    }
    # ... continue adding all other expenses
]

expenses_data.extend(more_expenses)

# Add all the expenses
bulk_add_expenses(expenses_data)

def format_indian_currency(amount):
    """Format amount in Indian Rupees with proper comma placement"""
    # Convert to string with 2 decimal places
    str_amount = f"{amount:,.2f}"
    
    # Split into whole and decimal parts
    parts = str_amount.split('.')
    whole = parts[0].replace(',', '')
    decimal = parts[1]
    
    # Add commas for Indian number system
    result = ''
    # First comma after 3 digits from right
    if len(whole) > 3:
        result = ',' + whole[-3:]
        whole = whole[:-3]
        # Then commas after every 2 digits
        while whole:
            if len(whole) > 2:
                result = ',' + whole[-2:] + result
                whole = whole[:-2]
            else:
                result = whole + result
                whole = ''
    else:
        result = whole
    
    # Add decimal part and currency symbol
    return f"₹{result}.{decimal}"

def analyze_expenses():
    """
    Analyze expenses and show summary statistics
    """
    if expenses_df.empty:
        print("No expenses data available.")
        return
        
    print("\nEXPENSE ANALYSIS")
    print("=" * 50)
    
    # Calculate total expenses
    total = expenses_df['Amount'].sum()
    print(f"\nTotal Expenses: {format_indian_currency(total)}")
    
    # Expenses by category
    print("\nExpenses by Category:")
    category_totals = expenses_df.groupby('Category')['Amount'].sum().sort_values(ascending=False)
    for category, amount in category_totals.items():
        percentage = (amount / total) * 100
        print(f"{category}: {format_indian_currency(amount)} ({percentage:.1f}%)")
    
    # Monthly summary
    print("\nMonthly Expense Summary:")
    monthly_expenses = expenses_df.set_index('Date').resample('ME')['Amount'].sum()
    for date, amount in monthly_expenses.items():
        print(f"{date.strftime('%B %Y')}: {format_indian_currency(amount)}")

def visualize_expenses():
    """
    Create visualizations for expense analysis
    """
    if expenses_df.empty:
        print("No expenses data available for visualization.")
        return
    
    # Create a pie chart for category distribution
    fig1 = px.pie(expenses_df, values='Amount', names='Category', 
                  title='Expense Distribution by Category',
                  hover_data=['Amount'],
                  custom_data=['Amount'])
    fig1.update_traces(hovertemplate='Category: %{label}<br>Amount: ₹%{customdata:,.2f}<br>Percentage: %{percent:.1%}')
    fig1.show()
    
    # Create a bar chart for subcategory distribution
    category_sub = expenses_df.groupby(['Category', 'Subcategory'])['Amount'].sum().reset_index()
    fig2 = px.bar(category_sub, x='Category', y='Amount', color='Subcategory',
                  title='Expenses by Category and Subcategory')
    fig2.update_layout(yaxis_title="Amount (₹)")
    fig2.update_traces(hovertemplate='Category: %{x}<br>Subcategory: %{customdata}<br>Amount: ₹%{y:,.2f}')
    fig2.show()
    
    # Create a line chart for cumulative expenses over time
    expenses_df_sorted = expenses_df.sort_values('Date')
    expenses_df_sorted['Cumulative'] = expenses_df_sorted['Amount'].cumsum()
    fig3 = px.line(expenses_df_sorted, x='Date', y='Cumulative',
                   title='Cumulative Expenses Over Time')
    fig3.update_layout(yaxis_title="Cumulative Amount (₹)")
    fig3.update_traces(hovertemplate='Date: %{x}<br>Cumulative Amount: ₹%{y:,.2f}')
    fig3.show()

def generate_monthly_report():
    """
    Generate a detailed monthly report of expenses
    """
    if expenses_df.empty:
        print("No expenses data available for report generation.")
        return
    
    current_month = pd.Timestamp.now().strftime('%B %Y')
    
    print(f"\nMONTHLY EXPENSE REPORT - {current_month}")
    print("=" * 50)
    
    # Get current month's expenses
    current_month_mask = expenses_df['Date'].dt.strftime('%B %Y') == current_month
    month_expenses = expenses_df[current_month_mask]
    
    if month_expenses.empty:
        print("\nNo expenses recorded for the current month.")
        return
    
    # Total expenses this month
    total = month_expenses['Amount'].sum()
    print(f"\nTotal Expenses This Month: {format_indian_currency(total)}")
    
    # Expenses by category this month
    print("\nExpenses by Category:")
    category_totals = month_expenses.groupby('Category')['Amount'].sum()
    for category, amount in category_totals.items():
        percentage = (amount / total) * 100
        print(f"{category}: {format_indian_currency(amount)} ({percentage:.1f}%)")
    
    # Detailed expense list
    print("\nDetailed Expense List:")
    for _, expense in month_expenses.iterrows():
        print("-" * 100)
        print(f"Date: {expense['Date'].strftime('%Y-%m-%d')}")
        print(f"Category: {expense['Category']} - {expense['Subcategory']}")
        print(f"Description: {expense['Description']}")
        print(f"Amount: {format_indian_currency(expense['Amount'])}")
        print(f"Payment Method: {expense['Payment_Method']}")
        if pd.notna(expense['Note']):
            print(f"Note: {expense['Note']}")

# Re-run the analysis with Indian currency formatting
print("Running analysis with Indian currency formatting...")
analyze_expenses()
visualize_expenses()
generate_monthly_report()

## 5. Usage Examples

Here are some examples of how to use the expense tracker:

In [ ]:
# Example 1: Adding a new expense
add_expense(
    date="2025-06-01",
    category="Foundation",
    subcategory="Concrete",
    description="Initial foundation concrete work",
    amount=15000,
    payment_method="Bank Transfer",
    receipt_reference="FOUND001"
)

# Example 2: Adding another expense
add_expense(
    date="2025-06-01",
    category="Permits",
    subcategory="Building Permit",
    description="Building permit application fee",
    amount=2500,
    payment_method="Credit Card",
    receipt_reference="PERM001"
)

# Continue adding the remaining expenses
remaining_expenses = [
    {
        'date': '2025-04-01',
        'category': 'Transportation',
        'subcategory': 'Permits',
        'description': 'Check post fees',
        'amount': 600,
        'note': 'check post cost'
    },
    {
        'date': '2025-04-01',
        'category': 'Transportation',
        'subcategory': 'Permits',
        'description': 'Transportation permit',
        'amount': 2600,
        'note': 'Permit for transportation of teak'
    },
    {
        'date': '2025-04-01',
        'category': 'Transportation',
        'subcategory': 'Personal Transport',
        'description': 'Travel expenses',
        'amount': 500,
        'note': 'General transportation to pormamilla'
    },
    {
        'date': '2025-04-01',
        'category': 'Planning',
        'subcategory': 'Other Expenses',
        'description': 'Poojari fees',
        'amount': 500,
        'note': 'Given to Poojari'
    },
    # Adding all remaining entries
    {
        'date': '2025-04-13',
        'category': 'Construction',
        'subcategory': 'Coolie',
        'description': 'Construction labor advance',
        'amount': 10000,
        'note': 'Advance for the Coolie, construction'
    }
]

# Add more entries following the same pattern...

# Add these expenses to the dataset
expenses_data.extend(remaining_expenses)

# Bulk add all expenses
bulk_add_expenses(expenses_data)

# Run analysis and generate reports
print("\nGenerating expense analysis...")
analyze_expenses()

print("\nCreating visualizations...")
visualize_expenses()

print("\nGenerating monthly report...")
generate_monthly_report()

Added new expense: Initial foundation concrete work - $15,000.00
Added new expense: Building permit application fee - $2,500.00

EXPENSE ANALYSIS

Total Expenses: $35,000.00

Expenses by Category:
Foundation: $30,000.00 (85.7%)
Permits: $5,000.00 (14.3%)

Monthly Expense Summary:
June 2025: $35,000.00


C:\Users\peruguyadav\AppData\Local\Temp\ipykernel_22624\1366325746.py:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_expenses = expenses_df.set_index('Date').resample('M')['Amount'].sum()


C:\Users\peruguyadav\AppData\Local\Temp\ipykernel_22624\1366325746.py:51: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.




MONTHLY EXPENSE REPORT - June 2025

Total Expenses This Month: $35,000.00

Expenses by Category:
Foundation: $30,000.00 (85.7%)
Permits: $5,000.00 (14.3%)

Detailed Expense List:
----------------------------------------------------------------------------------------------------
Date: 2025-06-01
Category: Foundation - Concrete
Description: Initial foundation concrete work
Amount: $15,000.00
Payment Method: Bank Transfer
Receipt Reference: FOUND001
----------------------------------------------------------------------------------------------------
Date: 2025-06-01
Category: Permits - Building Permit
Description: Building permit application fee
Amount: $2,500.00
Payment Method: Credit Card
Receipt Reference: PERM001
----------------------------------------------------------------------------------------------------
Date: 2025-06-01
Category: Foundation - Concrete
Description: Initial foundation concrete work
Amount: $15,000.00
Payment Method: Bank Transfer
Receipt Reference: FOUND001
--

In [ ]:
# Add the remaining expenses
final_expenses = [
    {
        'date': '2025-04-21',
        'category': 'Building Plan Approval',
        'subcategory': 'Property Tax',
        'description': 'Property tax payment',
        'amount': 8197,
        'note': 'property tax'
    },
    {
        'date': '2025-04-23',
        'category': 'Initial Setup',
        'subcategory': 'Basic Things',
        'description': 'Utility connections',
        'amount': 1000,
        'note': 'Electrical & water connections'
    },
    {
        'date': '2025-04-26',
        'category': 'Building Material',
        'subcategory': 'RR Stones',
        'description': 'Stone purchase',
        'amount': 4000,
        'note': '1 tractor'
    },
    {
        'date': '2025-04-26',
        'category': 'Initial Setup',
        'subcategory': 'Construction Equipment',
        'description': 'Construction tools',
        'amount': 2700,
        'note': 'bought 10 tubs, 2 paralu'
    },
    {
        'date': '2025-04-26',
        'category': 'Pooja',
        'subcategory': 'Bhumi Pooja',
        'description': 'Pooja items',
        'amount': 1530,
        'note': 'vastralu'
    },
    {
        'date': '2025-04-09',
        'category': 'Doors & Windows',
        'subcategory': 'Carpenter',
        'description': 'Carpenter advance',
        'amount': 4000,
        'note': 'Advance for Aachari'
    },
    # Add all remaining entries following this pattern...
    {
        'date': '2025-05-21',
        'category': 'Construction',
        'subcategory': 'Bricks',
        'description': 'Brick purchase',
        'amount': 6000,
        'note': 'fly ash bricks'
    }
]

# Add these expenses to the dataset
expenses_data.extend(final_expenses)

# Bulk add all expenses
bulk_add_expenses(expenses_data)

# Run final analysis
print("\nFinal Expense Analysis")
print("=" * 50)
analyze_expenses()
visualize_expenses()
generate_monthly_report()

In [ ]:
# Add all remaining expenses
complete_expenses = [
    {
        'date': '2025-04-28',
        'category': 'Construction',
        'subcategory': 'Coolie',
        'description': 'First payment after marking',
        'amount': 100000,
        'payment_method': 'Cash',
        'note': 'first payment to masthan, after marking'
    },
    {
        'date': '2025-04-28',
        'category': 'Pooja',
        'subcategory': 'Bhumi Pooja',
        'description': 'Pooja materials',
        'amount': 3000,
        'note': 'Bought things for Bhumi Pooja'
    },
    {
        'date': '2025-04-29',
        'category': 'Construction',
        'subcategory': 'Ramapuram Kankara',
        'description': 'Kankara before Bhumi Pooja',
        'amount': 23200,
        'note': '6 units kankara before bhumi pooja'
    },
    {
        'date': '2025-05-06',
        'category': 'Construction',
        'subcategory': 'JCB',
        'description': 'JCB work',
        'amount': 16000,
        'note': 'rent for JCB'
    },
    {
        'date': '2025-05-01',
        'category': 'Construction',
        'subcategory': 'kankara',
        'description': 'Kankara stones',
        'amount': 4800,
        'note': '1 tractor 40mm kankara'
    },
    # Add all the remaining transactions following this pattern
    {
        'date': '2025-05-29',
        'category': 'Construction',
        'subcategory': 'Iron',
        'description': 'Iron payment',
        'amount': 75000,
        'payment_method': 'PhonePe',
        'note': 'Iron ; phonepe'
    },
    {
        'date': '2025-05-29',
        'category': 'Construction',
        'subcategory': 'Cement',
        'description': 'Cement payment',
        'amount': 20000,
        'payment_method': 'PhonePe',
        'note': 'cement; Phonepe'
    }
]

# Add all expenses to the dataset
expenses_data.extend(complete_expenses)

# Final bulk addition of all expenses
bulk_add_expenses(expenses_data)

# Generate final reports
print("\nFinal Analysis with All Expenses")
print("=" * 50)
analyze_expenses()
visualize_expenses()
generate_monthly_report()

## 2. Define Expense Categories

Let's set up the categories and subcategories for tracking house construction expenses.